## Object detection and object counting in polygon zone: video file annotation

This notebook is an example how to use DeGirum PySDK to do object detection and object 
counting in polygon zone, annotating video file. The annotated video is saved into new file.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option.

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where you want run inference, video file name, model name, and other options here

In [ ]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# polygon_zones: zones in which objects need to be counted
# class_list: list of classes to be counted
# per_class_display: Boolean to specify if per class counts are to be displayed
# ann_path: path to save annotated video
hw_location = "@cloud"
model_zoo_url = "degirum/public"
model_name = "yolo_v5s_coco--512x512_quant_n2x_orca1_1"
video_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/Traffic.mp4"
polygon_zones = [
    [[265, 260], [730, 260], [870, 450], [120, 450]],
    [[400, 100], [610, 100], [690, 200], [320, 200]],
]
class_list = ["car", "motorbike", "truck"]
per_class_display = True
ann_path = "temp/object_in_zone_counting_video_file.mp4"

#### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg, degirum_tools

# load model
model = dg.load_model(
    model_name=model_name, 
    inference_host_address=hw_location,
    zoo_url=model_zoo_url,
    token=degirum_tools.get_token(),
    overlay_color=[(255,0,0)]
)

# create zone counter
zone_counter = degirum_tools.ZoneCounter(
    polygon_zones,
    class_list=class_list,
    per_class_display=per_class_display,
    triggering_position=degirum_tools.AnchorPoint.CENTER,
)

# attach zone counter to model
degirum_tools.attach_analyzers(model, [zone_counter])

# annotate video
degirum_tools.annotate_video(model, video_source, ann_path)

In [ ]:
# display result
degirum_tools.ipython_display(ann_path)

In [6]:
help(dg.model)

Help on module degirum.model in degirum:

NAME
    degirum.model

DESCRIPTION
    # model.py - DeGirum Python SDK: model implementation
    # Copyright DeGirum Corp. 2022
    #
    # Implements DeGirum model class
    #

CLASSES
    abc.ABC(builtins.object)
        Model
    
    class Model(abc.ABC)
     |  Model(model_name: str, model_params: degirum.aiclient.ModelParams, supported_device_types: List[str])
     |  
     |  Model class. Handles whole inference lifecycle for a single model: input data preprocessing, inference,
     |  and postprocessing.
     |  
     |  !!! note
     |  
     |      You never construct model objects yourself -- instead you call
     |      [degirum.zoo_manager.ZooManager.load_model][] method to create [degirum.model.Model][] instances for you.
     |  
     |  Method resolution order:
     |      Model
     |      abc.ABC
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __call__(self, data)
     |      Perform whole 